## Data Extraction Process

To build the analytical dataset, we combined multiple raw CSV sources from the NetEase Cloud Music (NCM) platform:

1) User Selection:
We first identify all users with registeredMonthCnt = 0 from user_demographics.csv. This group represents newly registered users, which are the focus of our analysis.
2) Impression Collection:
For these users, we extract all available impressions from impression_data.csv (no per-user cap). Each impression contains engagement signals such as clicks, likes, shares, and view times.
3) Dimensional Joins:
The impressions are enriched with static user, card (mlog), and creator metadata by joining:
user_demographics.csv on userId
mlog_demographics.csv on mlogId
creator_demographics.csv on creatorId
4) Behavioral Statistics:
Two additional datasets provide aggregated behavioral context:
mlog_stats.csv (joined on mlogId and dt) adds mlog-level engagement counts.
creator_stats.csv (joined on creatorId and dt) adds creator-level posting activity.

#### Post-processing and Output:
The merged dataset is cleaned, type-casted, and deduplicated. A binary label y_active is created, where users with user_level ≥ 5 are marked as active.
The final dataset is saved at the impression level (one row per impression event).

In [3]:
from pathlib import Path
import pandas as pd
import numpy as np

# ---------------------------
# Config
# ---------------------------
DATA_DIR = Path("../csv_data")         
OUT_DIR = Path("../data")
OUT_CSV = OUT_DIR / "recently_registered_users.csv"  

CHUNKSIZE = 250_000

# Raw file paths
IMP_PATH = DATA_DIR / "impression_data.csv"
USR_PATH = DATA_DIR / "user_demographics.csv"
CRD_PATH = DATA_DIR / "mlog_demographics.csv"
CRT_PATH = DATA_DIR / "creator_demographics.csv"
MLOG_STATS_PATH = DATA_DIR / "mlog_stats.csv"
CREATOR_STATS_PATH = DATA_DIR / "creator_stats.csv"

# ---------------------------
# Helpers
# ---------------------------
def users_with_registered_month_zero(user_csv, chunksize=CHUNKSIZE):
    """
    Return the set of userIds with registeredMonthCnt == 0.
    No sampling; scans the user_demographics.csv once in chunks.
    """
    keep = []
    usecols = ["userId", "registeredMonthCnt"]
    dtypes = {"userId": "string", "registeredMonthCnt": "Int64"}
    for chunk in pd.read_csv(user_csv, usecols=usecols, chunksize=chunksize, dtype=dtypes):
        sel = chunk.loc[chunk["registeredMonthCnt"].fillna(-1).eq(0), "userId"].dropna().astype("string")
        if not sel.empty:
            keep.append(sel)
    if not keep:
        return set()
    return set(pd.concat(keep, ignore_index=True).unique())

def read_filtered(path, key_col, keep_keys, usecols, chunksize=CHUNKSIZE):
    """
    Semi-join reader: only rows whose key_col ∈ keep_keys are returned.
    """
    keep_keys = {str(x) for x in pd.Series(list(keep_keys)).dropna().unique()}
    parts = []
    for chunk in pd.read_csv(path, usecols=usecols, chunksize=chunksize, dtype={key_col: "string"}):
        chunk[key_col] = chunk[key_col].astype("string")
        sel = chunk[chunk[key_col].isin(keep_keys)]
        if not sel.empty:
            parts.append(sel)
    return pd.concat(parts, ignore_index=True) if parts else pd.DataFrame(columns=usecols)

def read_filtered_two_keys(path, key_cols, keep_pairs, usecols, chunksize=CHUNKSIZE, dtypes=None):
    """
    Semi-join reader for composite key (k1, k2) using string-normalized composite keys.
    Works safely with pandas nullable dtypes (e.g., Int64Dtype).
    """
    k1, k2 = key_cols

    # --- build keep_set of "k1|k2" strings ---
    pairs_df = pd.DataFrame(keep_pairs, columns=[k1, k2]).dropna()
    if not pairs_df.empty:
        pairs_df[k1] = pairs_df[k1].astype("string")
        # force numeric then to nullable int for consistent stringification
        pairs_df[k2] = pd.to_numeric(pairs_df[k2], errors="coerce").astype("Int64")
        keep_set = set((pairs_df[k1].astype("string") + "|" + pairs_df[k2].astype("string")).tolist())
    else:
        keep_set = set()

    if dtypes is None:
        dtypes = {}

    parts = []
    for chunk in pd.read_csv(path, usecols=usecols, chunksize=chunksize, dtype=dtypes):
        # normalize keys in the chunk
        c1 = chunk[k1].astype("string")
        c2 = pd.to_numeric(chunk[k2], errors="coerce").astype("Int64")

        composite = (c1.astype("string") + "|" + c2.astype("string"))
        mask = composite.isin(keep_set)

        sel = chunk.loc[mask]
        if not sel.empty:
            parts.append(sel)

    return pd.concat(parts, ignore_index=True) if parts else pd.DataFrame(columns=usecols)


# ---------------------------
# Build impressions for selected users 
# ---------------------------
def collect_impressions_for_users(imp_path, selected_users, chunksize=CHUNKSIZE):
    usecols = [
        "userId","mlogId","impressTime","dt","impressPosition",
        "isClick","isLike","isComment","isShare","isViewComment",
        "isIntoPersonalHomepage","mlogViewTime"
    ]
    dtypes = {
        "userId": "string",
        "mlogId": "string",
        "dt": "Int16",
        "impressPosition": "Int16",
        "isClick": "Int8",
        "isLike": "Int8",
        "isComment": "Int8",
        "isShare": "Int8",
        "isViewComment": "Int8",
        "isIntoPersonalHomepage": "Int8",
        "mlogViewTime": "float32"
    }
    parts = []
    for chunk in pd.read_csv(imp_path, usecols=usecols, chunksize=chunksize, dtype=dtypes):
        sel = chunk[chunk["userId"].isin(selected_users)]
        if sel.empty:
            continue
        # Preserve early behavior ordering
        sel = chunk[chunk["userId"].isin(selected_users)].copy()
        sel.sort_values(["userId", "impressTime"], inplace=True, kind="mergesort")
        parts.append(sel)
    return pd.concat(parts, ignore_index=True) if parts else pd.DataFrame(columns=usecols)

# ---------------------------
# Post-merge hygiene
# ---------------------------
def cast_post_merge(df: pd.DataFrame) -> pd.DataFrame:
    if {"userId","mlogId","impressTime"}.issubset(df.columns):
        df.drop_duplicates(["userId","mlogId","impressTime"], inplace=True)

    for c in ["isClick","isLike","isComment","isShare","isViewComment","isIntoPersonalHomepage"]:
        if c in df:
            df[c] = df[c].fillna(0).astype("uint8")

    if "impressPosition" in df:
        df["impressPosition"] = df["impressPosition"].astype("Int16")
    if "mlogViewTime" in df:
        df["mlogViewTime"] = pd.to_numeric(df["mlogViewTime"], errors="coerce").astype("float32")

    for c in ["gender","province","type","contentId","talkId","creatorType"]:
        if c in df:
            df[c] = df[c].astype("category")

    return df

# ---------------------------
# Main build
# ---------------------------
def main():
    OUT_DIR.mkdir(parents=True, exist_ok=True)

    # 1) Explicitly select users with registeredMonthCnt == 0
    selected_users = users_with_registered_month_zero(USR_PATH, chunksize=CHUNKSIZE)
    print(f"Users with registeredMonthCnt = 0: {len(selected_users):,}")

    # 2) Pull ALL impressions for those users 
    imp = collect_impressions_for_users(IMP_PATH, selected_users, CHUNKSIZE)
    print(f"Collected impressions: {len(imp):,}")

    # 3) Keys for joins
    user_keys = imp["userId"].astype("string").unique()
    mlog_keys = imp["mlogId"].astype("string").unique()

    # 4) Read user/card/creator dims (static, leakage-free)
    users = read_filtered(
        USR_PATH, "userId", user_keys,
        ["userId","age","gender","province","level","registeredMonthCnt","followCnt"]
    )
    cards = read_filtered(
        CRD_PATH, "mlogId", mlog_keys,
        ["mlogId","type","contentId","talkId","publishTime","creatorId"]
    )
    creator_keys = cards["creatorId"].dropna().astype("string").unique() if not cards.empty else []
    creators = read_filtered(
        CRT_PATH, "creatorId", creator_keys,
        ["creatorId","creatorType","level"]
    )

    # 5) Merge core dims
    df = (imp
          .merge(users, on="userId", how="left", suffixes=("", "_user"))
          .merge(cards, on="mlogId", how="left", suffixes=("", "_card"))
          .merge(creators, on="creatorId", how="left", suffixes=("", "_creator"))
          )

    # 6) Rename levels and create label (as before)
    if "level" in df.columns:
        df.rename(columns={"level": "user_level"}, inplace=True)
    if "level_creator" in df.columns:
        df.rename(columns={"level_creator": "creator_level"}, inplace=True)
    elif "level_y" in df.columns and "creator_level" not in df.columns:
        df.rename(columns={"level_y": "creator_level"}, inplace=True)
    if "level_x" in df.columns and "user_level" not in df.columns:
        df.rename(columns={"level_x": "user_level"}, inplace=True)

    if "user_level" in df.columns:
        df["y_active"] = (pd.to_numeric(df["user_level"], errors="coerce") >= 2).astype("uint8")
    else:
        df["y_active"] = np.nan

    # 7) Merge mlog_stats on (mlogId, dt)
    if not df.empty:
        # Build pair keys from already-kept mlogId, dt
        mlog_dt_pairs = df[["mlogId", "dt"]].dropna().drop_duplicates().values.tolist()
        mlog_stats = read_filtered_two_keys(
            MLOG_STATS_PATH,
            key_cols=("mlogId", "dt"),
            keep_pairs=mlog_dt_pairs,
            usecols=[
                "mlogId","dt",
                "userImprssionCount","userClickCount","userLikeCount",
                "userCommentCount","userShareCount","userViewCommentCount",
                "userIntoPersonalHomepageCount","userFollowCreatorCount"
            ],
            dtypes={"mlogId": "string", "dt": "Int64"}
        )
        if not mlog_stats.empty:
            # Cast dt to Int16 to match df
            mlog_stats["dt"] = mlog_stats["dt"].astype("Int16")
            df = df.merge(mlog_stats, on=["mlogId","dt"], how="left", suffixes=("", "_mlogstats"))

    # 8) Merge creator_stats on (creatorId, dt)
    if not df.empty and "creatorId" in df.columns:
        creator_dt_pairs = df[["creatorId", "dt"]].dropna().drop_duplicates().values.tolist()
        creator_stats = read_filtered_two_keys(
            CREATOR_STATS_PATH,
            key_cols=("creatorId", "dt"),
            keep_pairs=creator_dt_pairs,
            usecols=["creatorId","dt","PushlishMlogCnt"],   # keep the source's column name
            dtypes={"creatorId": "string", "dt": "Int64"}
        )
        if not creator_stats.empty:
            creator_stats["dt"] = creator_stats["dt"].astype("Int16")
            df = df.merge(creator_stats, on=["creatorId","dt"], how="left", suffixes=("", "_creatorstats"))

    # 9) dtypes & hygiene
    df = cast_post_merge(df)

    # 10) Save
    df.to_csv(OUT_CSV, index=False)
    print(f"Saved: {OUT_CSV}")

if __name__ == "__main__":
    main()


Users with registeredMonthCnt = 0: 27,725
Collected impressions: 390,166
Saved: ../data/recently_registered_users.csv
